# Identify gaps in CARE service relative to unsheltered population
* Map number of CARE/CARE+ requests by tract (2017-2019)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps
import json
from ipywidgets import link, FloatSlider, Text, HTML
from branca.colormap import linear
import boto3

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'
s3 = boto3.client('s3')

In [3]:
df = gpd.read_file(f'{bucket_name}gis/intermediate/homelessness_care_tracts.geojson')
df.rename(columns = {'GEOID': 'id'}, inplace = True)
df = df.set_index('id')
df.head()

,SPA,SD,CD,year,unsheltered,sheltered,tot_homeless,bulky,encampment,illegal,other,pop,full_area,clipped_area,geometry
id,,,,,,,,,,,,,,,
06037139301,2,3,3,2017,16.156,0.0,16.156,1,28,1,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139301,2,3,3,2018,1.000,0.0,1.000,0,5,4,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139301,2,3,3,2019,4.315,0.0,4.315,6,54,0,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139302,2,3,3,2017,4.021,0.0,4.021,0,5,0,0,5389,0.130598,0.130598,"POLYGON ((-118.54073 34.18019, -118.54070 34.1..."
06037139302,2,3,3,2018,3.000,0.0,3.000,0,2,0,0,5389,0.130598,0.130598,"POLYGON ((-118.54073 34.18019, -118.54070 34.1..."


In [4]:
df.groupby('year').agg({'encampment': 'sum'}).reset_index()

,year,encampment
0,2017,13953
1,2018,21148
2,2019,26206


## Total CARE Requests

In [5]:
df1 = df.dissolve(by = ['id', 'clipped_area', 'full_area', 
                        'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])
# to_json converts the gdf to json. json.loads converts it into dictionary 
# geo_data can have many more columns than just geometry. Need to include other columns if you want to have the HTML popup include info.
geo_data = json.loads(df1.set_index('id').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
tot_care = dict(zip(df1['id'].tolist(), df1['encampment'].tolist()))

df1.encampment.describe()

count     994.000000
mean       61.677062
std       122.258584
min         0.000000
25%         7.000000
50%        25.000000
75%        67.750000
max      2351.000000
Name: encampment, dtype: float64

In [6]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = tot_care,
    colormap = linear.viridis,
    border_color = 'white',
    style = {'fillOpacity': 0.7, 'weight': 0.8, 'color': 'white', 'opacity': 0.8},
    hover_style = {'fillOpacity': 0.85},
    value_min = 0,
    value_max = 100
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Total CARE Requests:
        {} 
    '''.format(id, feature['properties']['encampment'])
 
    
layer.on_hover(update_html)



control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution':…

## CARE Requests per 100 residents

In [7]:
df2 = df1.copy()
df2['care_per100'] = df2.apply(lambda row: row['encampment'] / row['pop'] * 100 if row['pop'] != 0 else np.nan, axis = 1)

care_per100 = dict(zip(df2['id'].tolist(), df2['care_per100'].tolist()))

df2.care_per100.describe()

count     989.000000
mean        3.669005
std        46.814030
min         0.000000
25%         0.172911
50%         0.622483
75%         1.761543
max      1420.000000
Name: care_per100, dtype: float64

In [9]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = care_per100,
    colormap = linear.viridis,
    border_color = 'white',
    style = {'fillOpacity': 0.7, 'weight': 0.8, 'color': 'white', 'opacity': 0.8},
    hover_style = {'fillOpacity': 0.85}
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        CARE Requests per 100:
        {} 
    '''.format(id, feature['properties']['care_per100'])
 
    
layer.on_hover(update_html)



control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

ValueError: Thresholds are not sorted.